# Setup The Environment

In [ ]:
import numpy as np
import pandas as pd
import gzip
import json

from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error ,r2_score


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train_classification_df = pd.read_csv("/content/drive/MyDrive/train-classification.csv",)
train_classification_df = train_classification_df.rename(columns={'username': 'user_id', 'label': 'category'})

# Unifying labels
train_classification_df["category"] = train_classification_df["category"].apply(str.lower)
username2_category = train_classification_df.set_index("user_id").to_dict()["category"]

In [ ]:
train_data_path = "/content/drive/MyDrive/training-dataset.jsonl"

username2posts_train = dict()
username2profile_train = dict()

username2posts_test = dict()
username2profile_test = dict()


with open(train_data_path, "rt") as fh:
  for line in fh:
    sample = json.loads(line)

    profile = sample["profile"]
    username = profile["username"]

      # train data info
    username2posts_train[username] = sample["posts"]
    username2profile_train[username] = profile



# Functions used in Regression

In [ ]:
def predict_like_count(username, current_post=None):
  def get_avg_like_count(posts:list):
    total = 0.
    for post in posts:
      if current_post is not None and post["id"] == current_post["id"]:
        continue

      like_count = post.get("like_count", 0)
      if like_count is None:
        like_count = 0
      total += like_count

    if len(posts) == 0:
      return 0.

    return total / len(posts)

  if username in username2posts_train:
    return get_avg_like_count(username2posts_train[username])
  elif username in username2posts_test:
    return get_avg_like_count(username2posts_test[username])
  else:
    print(f"No data available for {username}")
    return -1

In [ ]:
def log_mse(y_true, y_pred):
  """
  Calculate the Log Mean Squared Error (Log MSE) for like counts (log(like_count + 1)).

  Parameters:
  - y_true: array-like, actual like counts
  - y_pred: array-like, predicted like counts

  Returns:
  - log_mse: float, Log Mean Squared Error
  """
  # Ensure inputs are numpy arrays
  y_true = np.array(y_true)
  y_pred = np.array(y_pred)

  # Log transformation: log(like_count + 1)
  log_y_true = np.log1p(y_true)
  log_y_pred = np.log1p(y_pred)

  # Compute squared errors
  squared_errors = (log_y_true - log_y_pred) ** 2

  # Return the mean of squared errors
  return np.mean(squared_errors)

In [ ]:
from datetime import datetime

def timestamp_to_int(timestamp_str):
    """
    Convert a timestamp string to a comparable integer (Unix timestamp).

    Parameters:
    - timestamp_str (str): The timestamp string in the format 'YYYY-MM-DD HH:MM:SS'.

    Returns:
    - int: Unix timestamp (seconds since epoch).
    """
    # Define the datetime format of the input string
    format_str = '%Y-%m-%d %H:%M:%S'

    # Parse the string into a datetime object
    dt_object = datetime.strptime(timestamp_str, format_str)

    # Convert the datetime object to a Unix timestamp
    unix_timestamp = int(dt_object.timestamp())

    return unix_timestamp


## KNN Regression
(did not gave a high accuracy)

In [ ]:
from sklearn import preprocessing as p
from sklearn.neighbors import KNeighborsRegressor
import xgboost as xg


In [ ]:
pred = []
real = []
# Create a list to store data for the DataFrame
like_count_and_time_data = []
result = []
for username, posts in username2posts_train.items():
  print("post changed")
  for guess in posts:
    if guess.get("like_count",-1) == -1:
      continue
    x = []
    y=[]
    for post in posts:

        if post["like_count"] != None and post.get("comments_count",-1) and post.get("timestamp",-1) != -1 and post["id"]!=guess["id"]:

          x.append([np.log(post["comments_count"]+1),timestamp_to_int(post["timestamp"]),np.log(predict_like_count(username, guess)+1)])
          if post["like_count"] != None:
            y.append(np.log(post["like_count"]+1))
          else:
            y.append(predict_like_count(username,guess))

#log transform
    x = np.array(x)
    min_max_scaler = p.MinMaxScaler()
    yscaler = p.MinMaxScaler()
    y = np.array(y)

    x = min_max_scaler.fit_transform(x)
    rf_model = KNeighborsRegressor(n_neighbors=4)
    if len(x) > 2:
      rf_model.fit(x, y)
      x_pred = np.array([[np.log(guess["comments_count"]+1),timestamp_to_int(guess["timestamp"]),np.log(predict_like_count(username, guess)+1)]])
      x_pred = min_max_scaler.fit_transform(x_pred)
      rf_predictions = rf_model.predict(x_pred)
      rf_predictions = np.exp(rf_predictions)

      try:

        pred.append(rf_predictions[0])
        real.append(guess["like_count"])
        #rf_r2 = r2_score(guess["like_count"], rf_predictions[0])
        #rf_r2=r2_score()

        #result.append(rf_r2)
        #print("score",rf_r2,"id:",guess["id"],"likes:",guess["like_count"],"predicted:",rf_predictions)

      except:
        print("error")
import numpy as np

print(np.mean(result))

post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

# GradientBoosting
(submitted model)

In [ ]:
#Log added and debug
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_iterative_imputer
import numpy as np

pred = []
real = []
# Create a list to store data for the DataFrame
like_count_and_time_data = []
result = []

for i, (username, posts) in enumerate(username2posts_train.items()):
    if i >= 60:
        break
    print(f"Processing user {i + 1}: {username}")

    for guess in posts:
        if guess.get("like_count", -1) == -1:
            continue

        x = []
        y = []

        for post in posts:
            if post.get("like_count", -1) != -1 and post.get("comments_count", -1) and post.get("timestamp", -1) != -1 and post["id"] != guess["id"]:
                x.append([post["comments_count"], timestamp_to_int(post["timestamp"]), np.log1p(predict_like_count(username, guess) + 1)])
                if post["like_count"] is not None:
                    y.append(np.log1p(post["like_count"] + 1))
                else:
                    y.append(np.log1p(predict_like_count(username, guess) + 1))

        # Ensure enough data for training
        if len(x) > 2:
            rf_model = GradientBoostingRegressor(n_estimators=50, max_depth=45, min_samples_split=5, random_state=42)
            rf_model.fit(x, y)  # Training on log-transformed y

            rf_predictions = rf_model.predict([[guess["comments_count"], timestamp_to_int(guess["timestamp"]),np.log1p(predict_like_count(username, guess) + 1)]])

            try:
                # Cap and handle overflow in expm1 for safety
                pred_value = np.expm1(rf_predictions[0]) - 1
                real_value = guess["like_count"]  # Cap like_count to avoid overflow

                pred.append(pred_value)
                real.append(real_value)

                # Debug log for values
                print(f"User: {username}, Post ID: {guess['id']}, Predicted: {pred_value}, Actual: {real_value}")

            except Exception as e:
                print(f"Error processing user {username}, Post ID: {guess['id']}: {e}")

# Calculate and display mean result
try:
    mean_result = np.mean(result) if result else float('nan')
    print(f"Mean result: {mean_result}")
except Exception as e:
    print(f"Error calculating mean result: {e}")


Processing user 1: deparmedya
User: deparmedya, Post ID: 17990918969458720, Predicted: 21.842662609794573, Actual: 6
User: deparmedya, Post ID: 18219250732221045, Predicted: 23.521627620382635, Actual: 22
User: deparmedya, Post ID: 18311380465102328, Predicted: 21.842662609794573, Actual: 19
User: deparmedya, Post ID: 18089518138361507, Predicted: 24.405446413967333, Actual: 19
User: deparmedya, Post ID: 18012743929758497, Predicted: 21.842662609794573, Actual: 21
User: deparmedya, Post ID: 17953423070513677, Predicted: 21.842662609794573, Actual: 24
User: deparmedya, Post ID: 18005686453762178, Predicted: 21.842662609794573, Actual: 11
User: deparmedya, Post ID: 17998285486955458, Predicted: 21.842662609794573, Actual: 7
User: deparmedya, Post ID: 17973191972514507, Predicted: 21.842662609794573, Actual: 6
User: deparmedya, Post ID: 17924398310711517, Predicted: 21.842662609794573, Actual: 9
User: deparmedya, Post ID: 17990108264053205, Predicted: 21.842662609794573, Actual: 26
User: 

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.experimental import enable_iterative_imputer

pred = []
real = []
x2 = []
# Create a list to store data for the DataFrame
like_count_and_time_data = []
result = []
counter = 0
for username, posts in username2posts_train.items():
  if counter>30:
    break
  counter+=1
  print("post changed")
  for guess in posts:
    if guess.get("like_count",-1) == -1:
      continue
    x = []
    y=[]
    for post in posts:

        if post.get("like_count",-1) != -1 and post.get("comments_count",-1) and post.get("timestamp",-1) != -1 and post["id"]!=guess["id"]:

          x.append([np.log(guess["comments_count"]+1),timestamp_to_int(guess["timestamp"]),np.log(predict_like_count(username, guess)+1)])
          if post["like_count"] != None:
            y.append(np.log(post["like_count"]+1))
          else:
            y.append(predict_like_count(username,guess))

#log transform
    x = np.array(x)
    rf_model = GradientBoostingRegressor(n_estimators=50, max_depth=10, min_samples_split=5, random_state=42)
    if len(x) > 2:
      rf_model.fit(x, y)
      rf_predictions = rf_model.predict([[guess["comments_count"],timestamp_to_int(guess["timestamp"]),predict_like_count(username, guess)]])

      try:
        x2.append(predict_like_count(username, guess))
        pred.append(rf_predictions[0])
        real.append(guess["like_count"])


      except:
        print("error")



post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed
post changed


# Evaluate model

In [ ]:
#checking validity of the output
import numpy as np
pred = np.asarray(pred)
real = np.asarray(real).astype(float)
print(np.isnan(pred).any())
print(np.isnan(real).any())
real = np.asarray(real)


False
False


In [ ]:
real = np.nan_to_num(real, nan=0)
pred = np.nan_to_num(pred, nan=0)
pred[pred<0]=0
r2_score(real,np.absolute(pred))

0.25727581656255694

In [ ]:
print(r2_score(real,pred))


0.17076764263834987


In [ ]:
print(log_mse(real,pred))


0.7564585801074245


In [ ]:
print(log_mse(real,np.absolute(pred)))
print(r2_score(real,pred))


0.702639408209792
0.113580593461856


In [ ]:
import numpy as np
np.average(result)

nan

# Model Submitted

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
counter = 0
def predictlike(username, guess):
  for username, posts in username2posts_train.items():
    username2posts_train[username] = posts

    x = []
    y=[]
    for post in posts:

        if post.get("like_count",-1) != -1 and post.get("comments_count",-1) and post.get("timestamp",-1) != -1 and post["id"]!=guess["id"]:

          x.append([post["comments_count"],timestamp_to_int(post["timestamp"]),predict_like_count(username, guess)])
          if post["like_count"] != None:
            y.append(post["like_count"])
          else:
            y.append(predict_like_count(username,guess))#add average to the dataset
        elif post["id"]==guess["id"] and post.get("like_count",-1) != -1:
          return post["like_count"]




    rf_model = GradientBoostingRegressor(n_estimators=50, max_depth=45, min_samples_split=5, random_state=42)
    if len(x) > 2:
      rf_model.fit(x, y)

      rf_predictions = rf_model.predict([[guess["comments_count"],timestamp_to_int(guess["timestamp"]),predict_like_count(username, guess)]])

      return abs(rf_predictions[0])

In [ ]:
import json  # Import the json module
from tqdm import tqdm

path = "/content/drive/MyDrive/test-regression-round2.jsonl"
output_path = "test-regression-round2.jsonl"

output_dict = {}
with open(path, "rt") as fh:
    for line in tqdm(fh):
        sample = json.loads(line)

        # Predict like count
        try:
          pred_val = predictlike(sample["username"],sample)
        except:
          print(sample["username"],sample)

        # Add post_id as key and predicted like count as value
        output_dict[sample["id"]] = round(float(pred_val))

# Write the output dictionary as JSON
with open(output_path, "wt") as of:
    json.dump(output_dict,of)

565it [00:28, 22.86it/s]

muratturkili {'caption': 'LOVE MY CITY #istanbul', 'comments_count': 0, 'id': '18000432757541225', 'media_type': 'IMAGE', 'media_url': 'https://scontent-sof1-2.cdninstagram.com/v/t51.29350-15/316585901_3321704488067846_8125311713018971247_n.jpg?_nc_cat=107&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=Z4jsF5_msc4AX_LO1Pc&_nc_ht=scontent-sof1-2.cdninstagram.com&edm=AL-3X8kEAAAA&oh=00_AfCmotPvme8GHxY8z9kxD6bPLfvCmdUI3KVFCoKGIKkDIw&oe=655B42C8', 'timestamp': '2022-11-24 08:35:55', 'username': 'muratturkili'}


2649it [02:11, 18.48it/s]

belediyesikose {'caption': 'Köse Belediye Resmi İnstagram Hesabıdır. \n🕖 7 / 24 - 📞 0456 411 50 62', 'comments_count': 1, 'id': '18071676892244749', 'media_type': 'IMAGE', 'media_url': 'https://scontent-sof1-1.cdninstagram.com/v/t51.29350-15/124388687_368892500856840_6008873979433304701_n.jpg?_nc_cat=104&ccb=1-7&_nc_sid=c4dd86&_nc_ohc=4bGCT5sIkRMAX8ObF_G&_nc_ht=scontent-sof1-1.cdninstagram.com&edm=AL-3X8kEAAAA&oh=00_AfDaQey6RNJoI8lpbXw28-DCkCZ09E5la0gd7LcbZkf65A&oe=6551DC65', 'timestamp': '2020-11-10 12:17:08', 'username': 'belediyesikose'}


3000it [02:29, 20.10it/s]
